# Exercise - Boosting for classification

1. Use the **load_breast_cancer** data (remember to split your data into a train, validation, and test data). Using your training and validation data, optimize the parameters of your GradientBoostingClassifier. How well does your optimized model perform on the test data?
1. Implement an RF and a SVM and use these as well (**note**: you may want to perform standardization for the SVM). How well do they perform on the test data? Try to "vote" using all three models (boosting, RF, and SVM) and select the class with the most votes. How well does your ensemble of all three models perform?

**See slides for more details!**

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import ensemble
import pandas as pd
import numpy as np

X, y = load_breast_cancer(return_X_y=True)
cancer_data = load_breast_cancer()
print(cancer_data.DESCR)

# Use `train_test_split` to split your data into a train and a test set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use `train_test_split` to split your train data into a train and a validation  set.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

# Exercise 1

Use the **load_breast_cancer** data (remember to split your data into a train, validation, and test data). Using your training and validation data, optimize the parameters of your GradientBoostingClassifier. How well does your optimized model perform on the test data?

Let us start by ensuring we can just run an GBT without any optimization.

In [3]:
gbt_current = ensemble.GradientBoostingClassifier()
gbt_current.fit(X_train, y_train)
y_val_hat = gbt_current.predict(X_val)
acc = accuracy_score(y_val, y_val_hat)

print(f'Boosting with default settings has validation accuracy of {round(acc * 100, 2)}%.')

Boosting with default settings has validation accuracy of 95.6%.


In [16]:
# Remember you can try other stuff than these specific parameters.
# Just here to get you started!

n_estimators_list = [100,200,300]
min_samples_split_list = [10,5,20]
min_samples_leaf_list = [15,10,8]

results = []

for n_estimators in n_estimators_list:
    for min_samples_split in min_samples_split_list:
        for min_samples_leaf in min_samples_leaf_list:
            gbt_current = ensemble.GradientBoostingClassifier(
                n_estimators=n_estimators,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                )
            gbt_current.fit(X_train, y_train)
            y_val_hat = gbt_current.predict(X_val)
            acc = accuracy_score(y_val, y_val_hat)

            results.append([acc, n_estimators, min_samples_split, min_samples_leaf])

results = pd.DataFrame(results)
results.columns = ['Accuracy', 'n_estimators', 'min_samples_split', 'min_samples_leaf']
print(results)

    Accuracy  n_estimators  min_samples_split  min_samples_leaf
0   0.945055           100                 10                15
1   0.945055           100                 10                10
2   0.945055           100                 10                 8
3   0.956044           100                  5                15
4   0.934066           100                  5                10
5   0.923077           100                  5                 8
6   0.945055           100                 20                15
7   0.923077           100                 20                10
8   0.934066           100                 20                 8
9   0.956044           200                 10                15
10  0.967033           200                 10                10
11  0.945055           200                 10                 8
12  0.956044           200                  5                15
13  0.956044           200                  5                10
14  0.934066           200              

In [19]:
# Extract best parameters.
n_estimators_optimal = results.loc[results['Accuracy'].idxmax()]['n_estimators'].astype(int)
min_samples_split_optimal = results.loc[results['Accuracy'].idxmax()]['min_samples_split'].astype(int)
min_samples_leaf_optimal = results.loc[results['Accuracy'].idxmax()]['min_samples_leaf'].astype(int)

print(f'n_estmators = {n_estimators_optimal}, min_samples_leaf = {min_samples_leaf_optimal}, min_samples_split = {min_samples_split_optimal}')

n_estmators = 200, min_samples_leaf = 10, min_samples_split = 10


In [21]:
# Initialize your final model
gbt_optimal = ensemble.RandomForestRegressor(
                n_estimators=200,
                min_samples_split=10,
                min_samples_leaf=10,
                )

# Use both training and validation data to fit it using np.concatenate (np.concatenate "stacks" the array like rbind in R)
X_train = np.concatenate((X_train,X_val))
y_train = np.concatenate((y_train,y_val))
#y_train
gbt_optimal.fit(X_train, y_train)

# Predict on test data
y_test_hat = gbt_current.predict(X_test)

# Obtain and check accuracy on test data
acc = accuracy_score(y_test, y_test_hat)
print(f'Boosting with optimal settings has a test accuracy of {round(acc * 100, 2)}%.')


Boosting with optimal settings has a test accuracy of 95.61%.


# Exercise 2

Implement an RF and a SVM and use these as well (**note**: you may want to perform standardization for the SVM). How well do they perform on the test data? Try to "vote" using all three models (boosting, RF, and SVM) and select the class with the most votes. How well does your ensemble of all three models perform?

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn import svm

# Scale your data
scaler = StandardScaler()
Z_train = scaler.fit_transform(X_train)
Z_val = scaler.transform(X_val)
Z_test = scaler.transform(X_test)

In [ ]:
# You may want to optimize the settings if you want.
# Then, you can do it here.
# You can/may want to do this both for the RF and the SVM.

In [ ]:
# Initialize your final models


# Use both training and validation data to fit them using np.concatenate (np.concatenate "stacks" the array like rbind in R)


# Predict on test data


# Obtain and check mse on test data. Is it lower or higher than the RF?


In [ ]:
# Finally combine your predictions
# (you do not have to change the code here, but you may want to try to improve beyond this method)

# WARNING: The below code for voting is only valid for 2 classes - DO NOT USE IT FOR CASES WITH MORE THAN 2 CLASSES
y_test_hat_combined = np.c_[y_test_hat_gbt, y_test_hat_rf, y_test_hat_svm]
y_test_hat_combined = np.round(np.sum(y_test_hat_combined, axis=1) / y_test_hat_combined.shape[1]).astype(int)

acc = accuracy_score(y_test, y_test_hat_combined)

print(f'Ensemble of boosting, RF, and SVM achieved test accuracy of {round(acc * 100, 2)}%.')